In [1]:
from model import *
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


sample_input = torch.randn(1, 100, 3, 480, 480)
sample_input = sample_input.permute(0, 2, 1, 3, 4)
classifier = C3DC()
out = classifier(sample_input)
print(out.shape)

torch.Size([96, 8192])


In [2]:
from dataLoader import *

labels_path = "../labels/train_labels/train_data.json"
sample_vids = "../../dissData/train_vids"

video_dataset = VideoDataset(sample_vids, labels_path, transform=None)

In [3]:
sample_index = 0
sample_frames, sample_label = video_dataset[sample_index]
print(sample_frames.shape)
print(sample_label)

torch.Size([100, 3, 480, 480])
0


In [4]:
fc = FullyConnected()
out_fc = fc(out)
score_reg = ScoreRegressor()
output = score_reg(out_fc)
print(output.shape)

torch.Size([96, 1])


In [7]:
print(output[1])

tensor([0.0015], grad_fn=<SelectBackward0>)


In [5]:
data_loader = DataLoader(video_dataset, batch_size=5, shuffle=True)
eteModel = EndToEndModel(classifier, fc,final_score_regressor=ScoreRegressor())

TypeError: ScoreRegressor.__init__() got an unexpected keyword argument 'input_size'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.AdamW(eteModel.parameters(), lr=0.00001)

num_epochs = 5

for epoch in range(num_epochs):
    for frames, labels in data_loader:
        frames = frames.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = eteModel(frames)
        classification_output = outputs['classification']
        final_score_output = outputs['final_score']

        # Calculate individual losses for classification and final score
        classification_loss = criterion(classification_output, labels.float().view(-1, 1))
        final_score_loss = criterion(final_score_output, labels.float().view(-1, 1))

        # Combine the losses with appropriate weights or other logic if needed
        loss = classification_loss + final_score_loss

        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')